In [2]:
# Data Management
import pandas as pd
import numpy as np
from pandas_datareader.data import DataReader
from ta import add_all_ta_features
from statsmodels.tsa.stattools import adfuller

# Data Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

# Supervised Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold

# Reporting
import matplotlib.pyplot as plt



In [3]:
# start_date = '2017-01-01'
# end_date= '2022-06-01'
# symbol = 'ETH-USD'
# # df = DataReader(name=symbol, data_source='yahoo', start=start_date, end=end_date)
# yf.pdr_override()
# df = pdr.get_data_yahoo('SPY', start='2022-10-24', end='2022-12-23')
# df.head()

In [4]:
df = pd.read_csv("SydneyHousePrices.csv")
print(f"Length of Data: {len(df)}")
df.head()

Length of Data: 199504


,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house


In [5]:
# Interpret Data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199504 entries, 0 to 199503
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Date        199504 non-null  object 
 1   Id          199504 non-null  int64  
 2   suburb      199504 non-null  object 
 3   postalCode  199504 non-null  int64  
 4   sellPrice   199504 non-null  int64  
 5   bed         199350 non-null  float64
 6   bath        199504 non-null  int64  
 7   car         181353 non-null  float64
 8   propType    199504 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 13.7+ MB


### Feature Engineering - Common Tasks

##### Handle Non-Numberical Data

In [8]:
# Count Unique items for suburd
suburb_text_unique = df["suburb"].unique()
print("Unique Suburb: ", len(suburb_text_unique))
print("Perform Label Encoding...")

Unique Suburb:  685
Perform Label Encoding...


In [9]:
# Count unique items for propType
propType_text_unique = df["propType"].unique()
print("Unique Prop Types: ", len(propType_text_unique))
print("Perform onehotencoding...")

Unique Prop Types:  8
Perform onehotencoding...


In [10]:
labelencoder = LabelEncoder()
encoded_suburbs = labelencoder.fit_transform(df["suburb"])
df["suburb_encoded"] = encoded_suburbs
df.head()

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType,suburb_encoded
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,22
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,22
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house,654
3,2019-05-28,4,Avalon Beach,2107,1530000,3.0,1,2.0,house,22
4,2019-05-22,5,Whale Beach,2107,8000000,5.0,4,4.0,house,654


In [11]:
# One Hot Encoding
onehot_encoded = pd.get_dummies(df["propType"], prefix="pt", drop_first = True)
df = df.join(onehot_encoded)
df.head(3)

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType,suburb_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,22,0,1,0,0,0,0,0
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,22,0,1,0,0,0,0,0
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house,654,0,1,0,0,0,0,0


### Set Target

In [12]:
df["TARGET"] = df["sellPrice"]
df.head(3)

,Date,Id,suburb,postalCode,sellPrice,bed,bath,car,propType,suburb_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse,TARGET
0,2019-06-19,1,Avalon Beach,2107,1210000,4.0,2,2.0,house,22,0,1,0,0,0,0,0,1210000
1,2019-06-13,2,Avalon Beach,2107,2250000,4.0,3,4.0,house,22,0,1,0,0,0,0,0,2250000
2,2019-06-07,3,Whale Beach,2107,2920000,3.0,3,2.0,house,654,0,1,0,0,0,0,0,2920000


### Remove Redundant Features

In [13]:
df_drop = df.copy()
df_drop.drop(columns=["Date", "Id", "suburb", "propType", "sellPrice"], inplace = True)
df_drop.head()

,postalCode,bed,bath,car,suburb_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse,TARGET
0,2107,4.0,2,2.0,22,0,1,0,0,0,0,0,1210000
1,2107,4.0,3,4.0,22,0,1,0,0,0,0,0,2250000
2,2107,3.0,3,2.0,654,0,1,0,0,0,0,0,2920000
3,2107,3.0,1,2.0,22,0,1,0,0,0,0,0,1530000
4,2107,5.0,4,4.0,654,0,1,0,0,0,0,0,8000000


### Check for NaN or Inf Values

In [14]:
is_null = df_drop.isnull().values.any()
is_inf = df_drop.isin([np.inf, -np.inf]).values.any()
print("Is Null: ", is_null)
print("Is Inf: ", is_inf)


Is Null:  True
Is Inf:  False


In [15]:
df_drop = df_drop.fillna(df_drop.mean())
df_drop.isnull().values.any()

False

##### Feature Scalling - Min Max Scalling

In [18]:
df_scalling = df_drop.copy()
mms = MinMaxScaler()
df_scalling.iloc[:] = mms.fit_transform(df_scalling)
df_scalling.head()

C:\Users\14699\AppData\Local\Temp\ipykernel_202892\962290187.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_scalling.iloc[:] = mms.fit_transform(df_scalling)


,postalCode,bed,bath,car,suburb_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse,TARGET
0,0.037179,0.030612,0.010204,0.025,0.032164,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000563
1,0.037179,0.030612,0.020408,0.075,0.032164,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.001048
2,0.037179,0.020408,0.020408,0.025,0.956140,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.001360
3,0.037179,0.020408,0.000000,0.025,0.032164,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000712
4,0.037179,0.040816,0.030612,0.075,0.956140,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.003725


##### Train Test Split

In [21]:
# Use Correct Dataset
is_deep_learning = False
df_tts = df_scalling.copy() if is_deep_learning else df_drop.copy()
df_tts.head()

,postalCode,bed,bath,car,suburb_encoded,pt_duplex/semi-detached,pt_house,pt_other,pt_terrace,pt_townhouse,pt_villa,pt_warehouse,TARGET
0,2107,4.0,2,2.0,22,0,1,0,0,0,0,0,1210000
1,2107,4.0,3,4.0,22,0,1,0,0,0,0,0,2250000
2,2107,3.0,3,2.0,654,0,1,0,0,0,0,0,2920000
3,2107,3.0,1,2.0,22,0,1,0,0,0,0,0,1530000
4,2107,5.0,4,4.0,654,0,1,0,0,0,0,0,8000000


In [24]:
# Split X and Y data
X = df_tts.iloc[:, :-1].values
y = df_tts.iloc[:, -1].values

In [27]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=1, shuffle=True)

#### Machine Learning

In [30]:
#Train Regressor
regressor = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0)
regressor.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, random_state=0)

In [32]:
# Make Prediction on Test Set
y_pred = regressor.predict(X_test)
y_pred = [round(x, 0) for x in y_pred]

In [35]:
# Chedck Accuracy
cv = RepeatedKFold(n_splits = 5, n_repeats=3, random_state=1)
n_scores = cross_val_score(regressor, X_train, y_train, scoring = "neg_mean_absolute_error", cv=cv, n_jobs = - 1, 
                           error_score = "raise")

In [37]:
# Report Performance
print("MAE Avg: ", abs(n_scores.mean()))
print("MAE Std: ", n_scores.std())

MAE Avg:  389118.30686148786
MAE Std:  20152.482105475094
